# End-to-End Demonstration: Google Classroom

*Goal*: run the Google Classroom Extractor and then upload the results into an `LMS` database.

## Software Requirements

1. Be sure to install Python 3.9; if you have multiple versions, make sure that the `python` command runs version 3.9.x. You can confirm your version by running `python --version` at a command prompt.
1. Microsoft SQL Server 2017 or 2019, in Windows or Linux.

## Getting Started

1. Confirm you have [poetry](https://python-poetry.org) installed (`poetry --version`).
1. Follow the [Google Classroom setup instructions](../google-classroom/README.md) in order to create a `service-account.json` file.
1. Follow the [notebook instructions](README.md) to install dependencies used by this notebook.
1. Create an `LMS` database in SQL Server.

In [3]:
# Load some utilities
from IPython.display import display, Markdown

# Setup logging
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

## Prepare Input Data

Update the variables in the next block as needed.

In [4]:
CLASSROOM_ACCOUNT = "admin@ibamonitoring.org"
START_DATE = "2020-08-17"
END_DATE = "2021-05-23"
LOG_LEVEL = "INFO"
OUTPUT_DIRECTORY = "gc-data"
DB_ENGINE = "mssql"
DB_SERVER = "localhost"
DB_NAME = "LMS"
DB_PORT = 1433

## Run the Google Classroom Extractor

In [6]:
from edfi_google_classroom_extractor.helpers.arg_parser import MainArguments as gc_args
from edfi_google_classroom_extractor import facade

arguments = gc_args(
    CLASSROOM_ACCOUNT,
    LOG_LEVEL,
    OUTPUT_DIRECTORY,
    START_DATE,
    END_DATE
)

facade.run(arguments)

INFO:edfi_google_classroom_extractor.facade:Starting Ed-Fi LMS Google Classroom Extractor
INFO:edfi_google_classroom_extractor.api.courses:Pulling course data
INFO:edfi_google_classroom_extractor.api.usage:Pulling usage data
ERROR:edfi_google_classroom_extractor.facade:An exception occurred while connecting to the API
Traceback (most recent call last):
  File "c:\source\oss\lms\demonstration\.venv\lib\site-packages\edfi_google_classroom_extractor\facade.py", line 60, in request
    return request_all(
  File "c:\source\oss\lms\demonstration\.venv\lib\site-packages\edfi_google_classroom_extractor\request.py", line 48, in request_all
    request_all_usage_as_df(
  File "c:\source\oss\lms\demonstration\.venv\lib\site-packages\edfi_google_classroom_extractor\api\usage.py", line 120, in request_all_usage_as_df
    usage_df: DataFrame = request_latest_usage_as_df(
  File "c:\source\oss\lms\demonstration\.venv\lib\site-packages\edfi_google_classroom_extractor\api\usage.py", line 72, in reques

## Run the Learning Management System Data Store Loader (LMS-DS-Loader)

The default setup below uses Windows integrated security. For username/password security, please review the commented-out code.

In [6]:
from edfi_lms_ds_loader.helpers.argparser import MainArguments as lms_args
from edfi_lms_ds_loader import loader_facade

arguments = lms_args(
    OUTPUT_DIRECTORY,
    DB_ENGINE,
    LOG_LEVEL
)
arguments.set_connection_string_using_integrated_security(
    DB_SERVER,
    DB_PORT,
    DB_NAME,
)
# For password auth, comment out the line above and uncomment this one:
# arguments.set_connection_string(
#     DB_SERVER,
#     DB_PORT,
#     DB_NAME,
#     USERNAME,
#     PASSWORD,
# )

loader_facade.run_loader(arguments)

INFO:edfi_lms_ds_loader.loader_facade:Begin loading files into the LMS Data Store (DS)...
INFO:edfi_lms_ds_loader.migrator:Begin database auto-migration...
INFO:edfi_lms_ds_loader.migrator:Done with database auto-migration.
INFO:edfi_lms_ds_loader.loader_facade:No sections loaded. Skipping assignments.
INFO:edfi_lms_ds_loader.loader_facade:No assignments loaded. Skipping assignment submissions.
INFO:edfi_lms_ds_loader.loader_facade:No sections loaded. Skipping section associations.
INFO:edfi_lms_ds_loader.loader_facade:No sections loaded. Skipping section associations.
INFO:edfi_lms_ds_loader.loader_facade:No sections loaded. Skipping section associations.
INFO:edfi_lms_ds_loader.loader_facade:Done loading files into the LMS Data Store.
